In [ ]:
import numpy as np
import pandas as pd
import os

def tokenize(text):
    """Converte o texto para minúsculas e separa em tokens."""
    return text.lower().split()

def build_vocab_from_saved(vocab_tokens, vocab_indices):
    """Reconstrói o vocabulário (token -> índice) a partir das listas salvas."""
    return {token: idx for token, idx in zip(vocab_tokens, vocab_indices)}

def vectorize(text, vocab):
    """Converte um texto em um vetor bag-of-words usando o vocabulário fornecido."""
    vec = np.zeros(len(vocab), dtype=np.float32)
    for token in tokenize(text):
        if token in vocab:
            vec[vocab[token] - 1] += 1  # -1, pois salvamos índices começando em 1
    return vec

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def predict_mlp(X, model_params):
    """Realiza a predição usando o modelo MLP treinado."""
    W1 = model_params["W1"]
    b1 = model_params["b1"]
    W2 = model_params["W2"]
    b2 = model_params["b2"]

    Z1 = np.dot(X, W1) + b1
    A1 = np.maximum(0, Z1)  # ReLU
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    preds = (A2 >= 0.5).astype(int)
    return preds

def main():
    # Carregar o modelo salvo
    model_path = "data/trained_mlp_model.npz"
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Modelo não encontrado: {model_path}")

    model_data = np.load(model_path, allow_pickle=True)
    W1 = model_data["W1"]
    b1 = model_data["b1"]
    W2 = model_data["W2"]
    b2 = model_data["b2"]
    vocab_tokens = model_data["vocab_tokens"].tolist()
    vocab_indices = model_data["vocab_indices"].tolist()

    model_params = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }
    vocab = build_vocab_from_saved(vocab_tokens, vocab_indices)
    print(f"Vocabulário carregado com {len(vocab)} tokens.")

    # Carregar o arquivo de inputs (dataset2_inputs.csv) com delimitador ";"
    input_file = "data/dataset2_inputs.csv"
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Arquivo de input não encontrado: {input_file}")

    df_input = pd.read_csv(input_file, delimiter=";", engine="python", on_bad_lines="skip")
    # Filtrar linhas cujo ID comece com "D2-"
    df_input = df_input[df_input["ID"].str.startswith("D2-")]
    ids = df_input["ID"].tolist()
    texts = df_input["Text"].astype(str).tolist()
    print(f"Dataset de input carregado com {len(texts)} textos válidos.")

    # Vetorizar os textos usando o vocabulário
    X_input = np.array([vectorize(txt, vocab) for txt in texts], dtype=np.float32)
    print("Shape de X_input:", X_input.shape)

    # Fazer predições
    preds = predict_mlp(X_input, model_params)
    predicted_labels = ["AI" if p == 1 else "Human" for p in preds.flatten()]

    # Salvar resultados
    df_output = pd.DataFrame({
        "ID": ids,
        "Predicted": predicted_labels
    })
    output_file = "data/dataset2_predictions.csv"
    df_output.to_csv(output_file, index=False, sep=";")
    print(f"Arquivo de predições gerado: {output_file}")
    print(df_output.head())

if __name__ == "__main__":
    main()